## 维度扩展

In [13]:
import numpy as np
import pickle
import scipy.sparse as sp

In [4]:
idx_i = np.random.randint(0, 2708)
idx_j = np.random.randint(0, 2708)
a =[idx_i, idx_j]
print(a)

[1354, 2418]


In [10]:
with open('/Users/wangjun/bearning_fault/Demo_GCN/data/cora/data.pkl', 'rb') as f:
    feas = pickle.load(f)
f.close()

In [14]:
adj = feas['adj']
print(isinstance(adj, sp.csr_matrix))

True


In [16]:
adj = adj.tocoo()
index = np.vstack((adj.row, adj.col))
print(index.T)

[[   0    8]
 [   0  435]
 [   0  544]
 ...
 [2705 1839]
 [2705 1840]
 [2705 2216]]


In [20]:
# 判断生成的边是否在原来的边中(在这里利用了广播机制)
print(index[:, None, :].shape)
print(index[:, None].shape)

(2, 1, 5102)
(2, 1, 5102)


In [27]:
index[:, None].shape
index.T[:, None].shape

(5102, 1, 2)

In [28]:
a - index.T[:, None]

array([[[ 1354,  2410]],

       [[ 1354,  1983]],

       [[ 1354,  1874]],

       ...,

       [[-1351,   579]],

       [[-1351,   578]],

       [[-1351,   202]]])

In [22]:
def ismember(a, b, tol=5):
    rows_close = np.all(np.round(a - b[:, None], tol) == 0, axis=-1)
    return np.any(rows_close)

In [24]:
ismember(a, index.T)

False

In [31]:
# 广播机制首先进行右对齐
x1 = np.random.randn(2, 4, 3)
x2 = np.random.randn(1, 3)
x3 = x1 * x2
print(x3.shape)

(2, 4, 3)


## sp.dia_matrix的用法

In [35]:
print(adj.diagonal().shape)
print(adj.diagonal()[np.newaxis, :].shape)
print(adj.diagonal()[np.newaxis, :][0])

(2708,)
(1, 2708)


In [37]:
## sp.dia_matirx的用法
data = np.array([[1, 2, 3]])
dia_mx = sp.dia_matrix((data, [0]), shape=(3, 3))
print(dia_mx.toarray())

[[1 0 0]
 [0 2 0]
 [0 0 3]]


In [38]:
## sp.dia_matirx的用法
data = np.array([[1, 2, 3]])
dia_mx = sp.dia_matrix((data, [1]), shape=(3, 3))
print(dia_mx.toarray())

[[0 2 0]
 [0 0 3]
 [0 0 0]]


## pos_weight

In [41]:
adj_sample = np.array([
    [0, 1, 0, 1],
    [1, 0, 1, 0],
    [0, 1, 0, 0],
    [1, 0, 0, 0]])

total_edge = adj_sample.shape[0] * adj_sample.shape[0]
actual_edge = adj_sample.sum()
pos_weight = (total_edge - actual_edge) / actual_edge
# 正负样本的比重
print(pos_weight)

1.6666666666666667


## Norm

In [43]:
norm = adj_sample.shape[0] * adj_sample.shape[0] / float((adj_sample.shape[0] * adj_sample.shape[0]- adj.sum()) ** 2)
print(norm)

6.185392607744513e-07
